In [1]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, asin, acos, atan
%reload_ext autoreload
%autoreload 2
plt.style.use('latex.mplstyle')

from point_configuration import PointConfiguration

In [13]:
from algorithms import reconstruct_mds
from algorithms import reconstruct_srls
from algorithms import reconstruct_acd
from algorithms import reconstruct_sdp
from algorithms import reconstruct_dwmds
import sys

def localization_error(point_estimate):
    err = np.linalg.norm(p.points[:n] - point_estimate[:n])**2/n
    return err

def first_point_error(point_estimate):
    err = np.linalg.norm(p.points[:1] - point_estimate[:1])**2
    return err

def try_method(errs_dict, failed, method, error_type, **kwargs):
    try: 
        xhat = execute_method(method, kwargs)
        err = error_type(xhat)
        errs_dict[method][it,i,j] = err
        return xhat, err
    except:
        print("Unexpected error:", sys.exc_info()[0])
        failed.append(method)
        return 0, 1e5
        
def execute_method(method, kwargs):
    if method == 'MDS':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, method='geometric')
    
    if method == 'MDSoptspace':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, 
                                   method='geometric', mask=weights, 
                                   completion='optspace', print_out=False)
    if method == 'MDSalternate':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, 
                                   method='geometric', mask=weights, 
                                   completion='alternate', print_out=False)
    if method == 'SDR':
        x_SDRold, EDMbest = reconstruct_sdp(noisy_edm, weights, lamda, p.points)
        ### Added to avoid strange "too large to be a matrix" error
        xhat = np.zeros((N, d))
        xhat[:,:] = x_SDRold
    if method == 'ACD':
        xhat, fs, err_edms, err_points = reconstruct_acd(noisy_edm, weights, X_0=x0.copy(), 
                                                          real_points=p.points)
    if method == 'dwMDS':
        X_bar = kwargs.get('X_bar', None)
        r = kwargs.get('r', None)
        xhat, costs = reconstruct_dwmds(noisy_edm, X0=x0.copy(), W=weights, n=n, 
                                       X_bar=X_bar, r=r)
    if method == 'SRLS':
        xhat = reconstruct_srls(noisy_edm, p.points, indices=range(n))
    return xhat
    
def create_weights(N, method='all', **kwargs):
    nmissing = kwargs.get('nmissing',0)
    
    weights = np.ones((N,N))
    weights[range(N), range(N)] = 0
    
    if method == 'none':
        return weights
    
    # create indices object to choose from
    elif method == 'all':
        all_indices = np.triu_indices(N,1)
    elif method == 'first':
        all_indices = [np.zeros(N-1).astype(np.int), np.arange(1,N).astype(np.int)]
    ntotal = len(all_indices[0])
    # randomly choose from indices and set to 0
    choice = np.random.choice(ntotal, nmissing, replace=False)
    chosen = [all_indices[0][choice], all_indices[1][choice]]
    weights[chosen] = 0
    weights[chosen[1],chosen[0]] = 0
    return weights
    
print(create_weights(9, method='first'))
print(create_weights(9, method='all'))

[[ 0.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  0.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  0.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  0.]]
[[ 0.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  0.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  0.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  0.]]


In [4]:
import copy
from basics import create_noisy_edm, get_edm
#from plots_cti import plot_point_sets_3d, plot_matrix

# Global settings
d = 3
lamda = 1000
square_size = 1

## Simulation A

In [5]:
N = 10
num_it = 30
name = 'A_missing'
m = 9
noises = np.logspace(-2,0.5,10) 
x0 = np.zeros((N, d)) # set by method_x0 
method_x0 = 'SDR'
method_weights = 'first'

errs = [[] for noise in noises]
errs_dict = {
    'noise':copy.deepcopy(errs),
    'MDS':copy.deepcopy(errs),
    'MDSalternate':copy.deepcopy(errs),
    'MDSoptspace':copy.deepcopy(errs),
    'SDR':copy.deepcopy(errs),
    'ACD':copy.deepcopy(errs),
    'dwMDS':copy.deepcopy(errs),
    'SRLS':copy.deepcopy(errs)}

In [15]:
num_it = 3
name = 'A_new'
n = 1
#ms = [5, 10, 20, 40]
ms = [5, 10]
noises = np.logspace(-3,-1,4) 
n_distances = np.array([3,4,5,7,10,15,20,30,40])

method_x0 = 'SDR'
method_weights = 'first'

errs = np.ones((num_it, len(noises), len(n_distances)))*-1.0
errs_dict = {
    #'noise':copy.deepcopy(errs),
    #'MDS':copy.deepcopy(errs),
    'MDSalternate':copy.deepcopy(errs),
    'MDSoptspace':copy.deepcopy(errs),
    'SDR':copy.deepcopy(errs),
    'ACD':copy.deepcopy(errs),
    'dwMDS':copy.deepcopy(errs),
    'SRLS':copy.deepcopy(errs)}

In [14]:
for m in ms:
    print('m =',m)
    errs_dict = {
        'MDSalternate':copy.deepcopy(errs),
        'MDSoptspace':copy.deepcopy(errs),
        'SDR':copy.deepcopy(errs),
        'ACD':copy.deepcopy(errs),
        'dwMDS':copy.deepcopy(errs),
        'SRLS':copy.deepcopy(errs)}
    N = m + n
    x0 = np.zeros((N, d)) # set by method_x0
    p = PointConfiguration(N, d)

    for it in range(num_it):
        print('it =',it)
        p.set_points('random',size=square_size)

        for i, noise in enumerate(noises):
            print('noise =', i)
            for j, n_dist in enumerate(n_distances[n_distances <= m]):
                print('n_dist',j)
                nmissing = m - n_dist
                # missing distances
                weights = create_weights(N, method=method_weights, nmissing=nmissing)
                # add noise
                noisy_edm = create_noisy_edm(p.edm, noise, n)
        
                failed = []
                for method in errs_dict.keys():
                    if method != 'noise':
                        if method == method_x0:
                            x0, err = try_method(errs_dict, failed, method, localization_error)
                        else:
                            __, err = try_method(errs_dict, failed, method, localization_error)

                if len(failed)>0:
                    print('exception occured in',failed)
                #else:
                    #for method in errs_dict.keys():
                        #print('error {}: {:2.2e}'.format(method, errs_dict[method][i][j][-1]))
    np.save('results/{}_errs_dict_{}.npy'.format(name, m),errs_dict)

m = 5
it = 0
noise = 0
n_dist 0
n_dist 1
Unexpected error: <class 'numpy.linalg.linalg.LinAlgError'>
exception occured in ['MDSoptspace']
n_dist 2
noise = 1
n_dist 0
n_dist 1
n_dist 2
noise = 2
n_dist 0
n_dist 1
n_dist 2
noise = 3
n_dist 0
n_dist 1
n_dist 2
it = 1
noise = 0
n_dist 0
smallest eigenvalue is zero
n_dist 1
smallest eigenvalue is zero
n_dist 2
smallest eigenvalue is zero
noise = 1
n_dist 0
smallest eigenvalue is zero
n_dist 1
Unexpected error: <class 'numpy.linalg.linalg.LinAlgError'>
smallest eigenvalue is zero
exception occured in ['MDSoptspace']
n_dist 2
smallest eigenvalue is zero
noise = 2
n_dist 0
smallest eigenvalue is zero
n_dist 1
Unexpected error: <class 'numpy.linalg.linalg.LinAlgError'>
smallest eigenvalue is zero
exception occured in ['MDSoptspace']
n_dist 2
smallest eigenvalue is zero
noise = 3
n_dist 0
smallest eigenvalue is zero
n_dist 1
smallest eigenvalue is zero
n_dist 2
smallest eigenvalue is zero
it = 2
noise = 0
n_dist 0
n_dist 1
Unexpected error: <cla

In [ ]:
n = N - m
n_edges = N*(N-1)
p = PointConfiguration(N, d)

for it in range(num_it):
    print(it)
    p.set_points('random',size=square_size)
    
    for i, noise in enumerate(noises):
        # missing distances
        weights = create_weights(N, method=method_weights)
        
        failed = []
        noisy_edm = create_noisy_edm(p.edm, noise, n)
        err_noise = np.linalg.norm(np.sqrt(p.edm) - np.sqrt(noisy_edm))/n_edges**0.5
        errs_dict['noise'][i].append(err_noise)
        
        for method in errs_dict.keys():
            if method != 'noise':
                if method == method_x0:
                    x0 = try_method(errs_dict, failed, method, localization_error)
                else:
                    try_method(errs_dict, failed, method, localization_error)
        
        if len(failed)>0:
            print('exception occured in',failed)
        else:
            for method in errs_dict.keys():
                print('error {}: {:2.2e}'.format(method, errs_dict[method][i][-1]))
np.save('results/{}_errs_dict.npy'.format(name),errs_dict)

## Simulation B

In [ ]:
num_it = 1
name = 'B_complete'
m = 5
#noises = np.logspace(-2,0.5,10) 
noises = np.logspace(-2,-1,1) 
x0 = np.zeros((N, d)) # set by method_x0 
method_x0 = 'SDR'
method_weights = 'none'
edm_noise = 1.0

In [ ]:
rs = np.linspace(0,10,15)

p = PointConfiguration(N, d)
weights = create_weights(N, method=method_weights)

errs = [[] for r in rs]
errs_dict = {'dwMDS':copy.deepcopy(errs),
             'SDR':copy.deepcopy(errs)}

for i, r_factor in enumerate(rs):
    for trials in range(20):
        p.set_points('random',size=square_size)
        noisy_edm = create_noisy_edm(p.edm, edm_noise, n)
        p.points[:n, :] += np.random.normal(0, noise, (n, p.d))
        X_bar = p.points[:n] 

        r = r_factor * np.ones((n,))
        r[0] = 0.0
        x0, err = try_method(errs_dict, failed, 'SDR', first_point_error)
        try_method(errs_dict, failed, 'dwMDS', first_point_error, X_bar=X_bar, r=r)

plt.figure()
for key, val in errs_dict.items():
    val_arr = np.array(val)
    val_arr = np.mean(val_arr, axis=1)
    plt.plot(rs, val_arr, label=key)
plt.legend(loc='best')
plt.show()

In [ ]:
r_factor=3. #value of all elements of r for dwMDS

errs = [[] for noise in noises]
errs_dict = {
    'noise':copy.deepcopy(errs),
    'MDS':copy.deepcopy(errs),
    'MDSalternate':copy.deepcopy(errs),
    'MDSoptspace':copy.deepcopy(errs),
    'SDR':copy.deepcopy(errs),
    'ACD':copy.deepcopy(errs),
    'dwMDS':copy.deepcopy(errs),
    'SRLS':copy.deepcopy(errs)}

n = N - m
n_edges = N*(N-1)
p = PointConfiguration(N, d)

for it in range(num_it):
    print(it)
    p.set_points('random',size=square_size)

    for i, noise in enumerate(noises):
        weights = create_weights(N, method=method_weights)

        # add noise to points.
        p.points[:n, :] += np.random.normal(0, noise, (n, p.d))

        # arguments for dwMDS: 
        X_bar = p.points[:n] 
        r = r_factor * np.ones((n,))
        r[0] = 0.0

        failed = []
        noisy_edm = create_noisy_edm(p.edm, edm_noise, n)
        err_noise = np.linalg.norm(np.sqrt(p.edm) - np.sqrt(noisy_edm))/n_edges**0.5
        errs_dict['noise'][i].append(err_noise)

        for method in errs_dict.keys():
            if method != 'noise':
                if method == method_x0:
                    x0, err = try_method(errs_dict, failed, method, first_point_error)
                else:
                    __, err = try_method(errs_dict, failed, method, first_point_error,
                                        X_bar=X_bar, r=r)

        if len(failed)>0:
            print('exception occured in',failed)
        else:
            for method in errs_dict.keys():
                print('error {}: {:2.2e}'.format(method, errs_dict[method][i][-1]))
np.save('results/{}_errs_dict.npy'.format(name),errs_dict)